##Ensemble learning

In [ ]:
from torchvision.models import vgg16
## Standard Library
import os

## External Libraries
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable
import torch.nn.functional as functional
from torch.utils.data import Dataset, DataLoader
from skimage import io
import matplotlib.pyplot as plt
from torchvision.transforms import functional as F
import pandas as pd
import torch.utils.data as data
from PIL import Image
import scipy.io as sio
import h5py
from torchvision import datasets
import torchvision

In [ ]:
!git clone https://github.com/ryanchankh/redunet_demo
%cd redunet_demo

Cloning into 'redunet_demo'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 77 (delta 26), reused 63 (delta 16), pack-reused 0
Unpacking objects: 100% (77/77), done.
/content/redunet_demo


In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print("Mounting Failed.")

Mounted at /content/gdrive


In [ ]:
class Loader_validation(data.Dataset):
	def __init__(self, path, transform=None):
		self.path = path
		self.dataset = datasets.ImageFolder(path, transform=transform)
		self.length = self.dataset.__len__()
		self.transform = transform
		
	def __getitem__(self, idx):
		data, y_task = self.dataset.__getitem__(idx)
				
		return data, torch.tensor(y_task).long().squeeze()

	def __len__(self):
		return self.length

class Loader_unif_sampling3(data.Dataset):
	def __init__(self, path1, path2, path3, transform=None):
		self.path_1 = path1
		self.path_2 = path2
		self.path_3 = path3
		
		self.dataset_1 = datasets.ImageFolder(self.path_1, transform=transform)
		self.dataset_2 = datasets.ImageFolder(self.path_2, transform=transform)
		self.dataset_3 = datasets.ImageFolder(self.path_3, transform=transform)
		
		self.len_1 = self.dataset_1.__len__()
		self.len_2 = self.dataset_2.__len__()
		self.len_3 = self.dataset_3.__len__()
		
		self.length = np.max([self.len_1, self.len_2, self.len_3])
		
		self.transform = transform

	def __getitem__(self, idx):

		idx_1 = idx % self.len_1
		idx_2 = idx % self.len_2
		idx_3 = idx % self.len_3

		data_1, y_task_1 = self.dataset_1.__getitem__(idx_1)
		
		data_2, y_task_2 = self.dataset_2.__getitem__(idx_2)
		
		data_3, y_task_3 = self.dataset_3.__getitem__(idx_3)	
				
		return data_1, torch.tensor(y_task_1).long().squeeze(), data_2, torch.tensor(y_task_2).long().squeeze(), data_3, torch.tensor(y_task_3).long().squeeze()

	def __len__(self):
		return self.length
  
class Loader_unif_sampling2(data.Dataset):
	def __init__(self, path1, path2, transform=None):
		self.path_1 = path1
		self.path_2 = path2
		
		self.dataset_1 = datasets.ImageFolder(self.path_1, transform=transform)
		self.dataset_2 = datasets.ImageFolder(self.path_2, transform=transform)
		
		self.len_1 = self.dataset_1.__len__()
		self.len_2 = self.dataset_2.__len__()
		
		self.length = np.max([self.len_1, self.len_2])
		
		self.transform = transform

	def __getitem__(self, idx):

		idx_1 = idx % self.len_1
		idx_2 = idx % self.len_2

		data_1, y_task_1 = self.dataset_1.__getitem__(idx_1)
		
		data_2, y_task_2 = self.dataset_2.__getitem__(idx_2)
				
		return data_1, torch.tensor(y_task_1).long().squeeze(), data_2, torch.tensor(y_task_2).long().squeeze()

	def __len__(self):
		return self.length

In [ ]:
if __name__ == '__main__':

    source_1 = '/content/gdrive/MyDrive/VLCS/CALTECH/full/'
    source_2 = '/content/gdrive/MyDrive/VLCS/LABELME/full/'
    source_3 = '/content/gdrive/MyDrive/VLCS/SUN/full/'
    source_4 = '/content/gdrive/MyDrive/VLCS/PASCAL/full/'
    img_transform = transforms.Compose([transforms.RandomResizedCrop(224, scale=(0.0, 0.0)),transforms.ToTensor()])#, transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    train_dataset4 = Loader_unif_sampling2(path1=source_2, path2=source_3, transform=img_transform)
    train_loader4 = torch.utils.data.DataLoader(dataset=train_dataset4, batch_size=32, shuffle=True, num_workers=0)
    train_dataset5 = Loader_unif_sampling2(path1=source_2, path2=source_4, transform=img_transform)
    train_loader5 = torch.utils.data.DataLoader(dataset=train_dataset5, batch_size=32, shuffle=True, num_workers=0)
    train_dataset6 = Loader_unif_sampling2(path1=source_3, path2=source_4, transform=img_transform)
    train_loader6 = torch.utils.data.DataLoader(dataset=train_dataset6, batch_size=32, shuffle=True, num_workers=0)
    train_dataset7 = Loader_unif_sampling3(path1=source_2, path2=source_3, path3=source_4, transform=img_transform)
    train_loader7 = torch.utils.data.DataLoader(dataset=train_dataset7, batch_size=32, shuffle=True, num_workers=0)
    # a, b, c, a_task, b_task, c_task = train_dataset7.__getitem__(2000)
    print("Done!")

Done!


In [ ]:
class Loader_validation(data.Dataset):
	def __init__(self, path, transform=None):
		self.path = path
		self.dataset = datasets.ImageFolder(path, transform=transform)
		self.length = self.dataset.__len__()
		self.transform = transform
		
	def __getitem__(self, idx):
		data, y_task = self.dataset.__getitem__(idx)
				
		return data, torch.tensor(y_task).long().squeeze()

	def __len__(self):
		return self.length

class Loader_test_sampling1(data.Dataset):
	def __init__(self, path, transform=None):
		self.path = path
		
		self.dataset = datasets.ImageFolder(self.path, transform=transform)
		
		self.length = self.dataset.__len__()
		
		self.transform = transform

	def __getitem__(self, idx):

		data, y_task = self.dataset.__getitem__(idx)
				
		return data, torch.tensor(y_task).long().squeeze()

	def __len__(self):
		return self.length

In [ ]:
if __name__ == '__main__':

    source_1 = '/content/gdrive/MyDrive/VLCS/CALTECH/full/'
    source_2 = '/content/gdrive/MyDrive/VLCS/LABELME/full/'
    source_3 = '/content/gdrive/MyDrive/VLCS/SUN/full/'
    source_4 = '/content/gdrive/MyDrive/VLCS/PASCAL/full/'
    img_transform = transforms.Compose([transforms.RandomResizedCrop(224, scale=(0.0, 0.0)), transforms.ToTensor()])#, transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    train_dataset1 = Loader_test_sampling1(path=source_2, transform=img_transform)
    train_loader1 = torch.utils.data.DataLoader(dataset=train_dataset1, batch_size=32, shuffle=True, num_workers=0)
    train_dataset2 = Loader_test_sampling1(path=source_3, transform=img_transform)
    train_loader2 = torch.utils.data.DataLoader(dataset=train_dataset2, batch_size=32, shuffle=True, num_workers=0)
    train_dataset3 = Loader_test_sampling1(path=source_4, transform=img_transform)
    train_loader3 = torch.utils.data.DataLoader(dataset=train_dataset3, batch_size=32, shuffle=True, num_workers=0)
    test_dataset = Loader_validation(path=source_1, transform=img_transform)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=32, shuffle=True, num_workers=0)
    a, a_task = train_dataset1.__getitem__(20)	
    b, b_task = train_dataset2.__getitem__(20)
    c, c_task = train_dataset3.__getitem__(20)
    print(a.size(), a_task)
    print(b.size(), b_task)
    print(c.size(), c_task)

torch.Size([3, 224, 224]) tensor(0)
torch.Size([3, 224, 224]) tensor(1)
torch.Size([3, 224, 224]) tensor(0)


In [ ]:
learning_rate = 0.01

model_vgg_11 = vgg16(pretrained=True) 
for param in model_vgg_11.parameters():
    param.requires_grad = False 
    model_vgg_11.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_11.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_11.parameters(), lr=learning_rate)

test_accuracies11 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs, labels) in enumerate(train_loader1):
        # print('test')
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()

        optimizer.zero_grad()
        outputs = model_vgg_11(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):   
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_11(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies11.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.75934636592865
10 :  0.9541839957237244
20 :  0.635023832321167
30 :  0.8547105193138123
40 :  0.8196662068367004
50 :  1.0410083532333374
60 :  1.0077208280563354
70 :  1.1239553689956665
80 :  0.8845750093460083
Test accuracy:  0.8190812468528748

EPOCH 2 of 10

0 :  0.8696197271347046
10 :  1.065382480621338
20 :  0.725410521030426
30 :  0.7223739624023438
40 :  0.5421465039253235
50 :  0.678015410900116
60 :  0.7393988370895386
70 :  0.6524592638015747
80 :  0.4662613868713379
Test accuracy:  0.8268551230430603

EPOCH 3 of 10

0 :  0.9668418169021606
10 :  0.8285807371139526
20 :  0.6216831803321838
30 :  0.8268712759017944
40 :  0.5561460256576538
50 :  0.7562382817268372
60 :  0.8325307965278625
70 :  0.8129671216011047
80 :  0.6792618632316589
Test accuracy:  0.8021201491355896

EPOCH 4 of 10

0 :  0.5877959132194519
10 :  0.6286115050315857
20 :  0.5692907571792603
30 :  0.5789539217948914
40 :  0.5603575706481934
50 :  1.088671088218689
60 :  0.800184905

In [ ]:
learning_rate = 0.01

model_vgg_22 = vgg16(pretrained=True) 
for param in model_vgg_22.parameters():
    param.requires_grad = False 
    model_vgg_22.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_22.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_22.parameters(), lr=learning_rate)

test_accuracies22 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs, labels) in enumerate(train_loader2):
        # print('test')
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()

        optimizer.zero_grad()
        outputs = model_vgg_22(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):   
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_22(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies22.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.7969532012939453
10 :  0.8848888874053955
20 :  0.7159256339073181
30 :  0.6406344771385193
40 :  0.728184163570404
50 :  1.0113911628723145
60 :  0.674847424030304
70 :  0.7656779885292053
80 :  0.7977070808410645
90 :  0.45137128233909607
100 :  0.8531884551048279
Test accuracy:  0.46219080686569214

EPOCH 2 of 10

0 :  0.6223695874214172
10 :  0.44186216592788696
20 :  0.8679604530334473
30 :  0.8046002984046936
40 :  0.8860998749732971
50 :  0.5723801851272583
60 :  0.4828554391860962
70 :  0.5461722612380981
80 :  0.6622391939163208
90 :  0.5632816553115845
100 :  0.4896881580352783
Test accuracy:  0.7123674750328064

EPOCH 3 of 10

0 :  0.5968760251998901
10 :  0.3978632092475891
20 :  0.3417462408542633
30 :  0.4430849552154541
40 :  0.6518605351448059
50 :  0.4837298095226288
60 :  0.8106622695922852
70 :  0.4761374294757843
80 :  0.7252277135848999
90 :  0.586745023727417
100 :  0.5318372249603271
Test accuracy:  0.7088339328765869

EPOCH 4 of 10

0 :  0

In [ ]:
learning_rate = 0.01

model_vgg_33 = vgg16(pretrained=True) 
for param in model_vgg_33.parameters():
    param.requires_grad = False 
    model_vgg_33.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_33.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_33.parameters(), lr=learning_rate)

test_accuracies33 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs, labels) in enumerate(train_loader3):
        # print('test')
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()

        optimizer.zero_grad()
        outputs = model_vgg_33(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):   
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_33(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies33.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.6319409608840942
10 :  0.8738385438919067
20 :  0.6096975207328796
30 :  0.8685361742973328
40 :  0.5780734419822693
50 :  0.5756531953811646
60 :  0.9617500305175781
70 :  0.7762264013290405
80 :  0.4641885757446289
90 :  0.6534415483474731
100 :  0.7388151288032532
Test accuracy:  0.9597172737121582

EPOCH 2 of 10

0 :  0.5773671269416809
10 :  0.5750580430030823
20 :  0.5881572365760803
30 :  0.41075068712234497
40 :  0.5148723125457764
50 :  0.8029566407203674
60 :  0.3698500394821167
70 :  0.35163214802742004
80 :  0.5905994772911072
90 :  0.620053768157959
100 :  0.9601091742515564
Test accuracy:  0.9420494437217712

EPOCH 3 of 10

0 :  0.7939535975456238
10 :  0.5695253014564514
20 :  0.5515101552009583
30 :  0.47286197543144226
40 :  0.763585090637207
50 :  0.7655980587005615
60 :  0.671181857585907
70 :  0.48959940671920776
80 :  0.6475889086723328
90 :  0.7593497037887573
100 :  0.3189297914505005
Test accuracy:  0.9696112871170044

EPOCH 4 of 10

0 :  

In [ ]:
learning_rate = 0.01

model_vgg_44 = vgg16(pretrained=True) 
for param in model_vgg_44.parameters():
    param.requires_grad = False 
    model_vgg_44.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_44.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_44.parameters(), lr=learning_rate)

test_accuracies44 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2) in enumerate(train_loader4):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_44(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_44(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_44(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies44.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.5813969373703003
10 :  0.7296568751335144
20 :  0.7042942643165588
30 :  0.6290290355682373
40 :  0.6124317049980164
50 :  0.5685247778892517
60 :  0.5304080843925476
70 :  0.4911779463291168
80 :  1.0438838005065918
90 :  0.6378762722015381
100 :  0.7406148314476013
Test accuracy:  0.7999999523162842

EPOCH 2 of 10

0 :  0.5589027404785156
10 :  0.632250189781189
20 :  0.78792405128479
30 :  0.8821187019348145
40 :  0.758495032787323
50 :  0.9298988580703735
60 :  0.5277863144874573
70 :  0.7246214151382446
80 :  0.744167149066925
90 :  0.705268383026123
100 :  0.7925928235054016
Test accuracy:  0.8063604235649109

EPOCH 3 of 10

0 :  0.635172963142395
10 :  0.5500786304473877
20 :  0.6829027533531189
30 :  0.5823062062263489
40 :  0.5981357097625732
50 :  0.6813749074935913
60 :  0.6923967599868774
70 :  1.0370829105377197
80 :  0.7943114638328552
90 :  0.8631778955459595
100 :  0.775212287902832
Test accuracy:  0.8070670962333679

EPOCH 4 of 10

0 :  0.6353283

In [ ]:
learning_rate = 0.01

model_vgg_55 = vgg16(pretrained=True) 
for param in model_vgg_55.parameters():
    param.requires_grad = False 
    model_vgg_55.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_55.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_55.parameters(), lr=learning_rate)

test_accuracies55 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2) in enumerate(train_loader5):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_55(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_55(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_55(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies55.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.5953596830368042
10 :  0.9245277643203735
20 :  0.4810572564601898
30 :  0.7956580519676208
40 :  0.6866698265075684
50 :  0.9904444217681885
60 :  0.5453447699546814
70 :  0.7377908825874329
80 :  0.7484648823738098
90 :  0.6615406274795532
100 :  0.3901725113391876
Test accuracy:  0.9335688948631287

EPOCH 2 of 10

0 :  0.5165274143218994
10 :  0.5376273393630981
20 :  0.9173305034637451
30 :  0.3379229009151459
40 :  0.8424703478813171
50 :  0.4639112949371338
60 :  0.40912774205207825
70 :  0.7672798037528992
80 :  0.5362766981124878
90 :  0.42754822969436646
100 :  0.8365994691848755
Test accuracy:  0.9554769992828369

EPOCH 3 of 10

0 :  0.5391398668289185
10 :  0.7167620062828064
20 :  0.694274365901947
30 :  0.9354164600372314
40 :  0.6522634625434875
50 :  0.6250236630439758
60 :  0.6634135246276855
70 :  0.48910588026046753
80 :  0.5274233818054199
90 :  0.6106094121932983
100 :  0.6758840084075928
Test accuracy:  0.9498233199119568

EPOCH 4 of 10

0 : 

In [ ]:
learning_rate = 0.01

model_vgg_66 = vgg16(pretrained=True) 
for param in model_vgg_66.parameters():
    param.requires_grad = False 
    model_vgg_66.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_66.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_66.parameters(), lr=learning_rate)

test_accuracies66 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2) in enumerate(train_loader6):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_66(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_66(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_66(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies66.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.452669620513916
10 :  1.0741939544677734
20 :  0.7699269652366638
30 :  0.44783666729927063
40 :  1.0120182037353516
50 :  0.7802507281303406
60 :  0.8664816617965698
70 :  0.715819239616394
80 :  0.4424489140510559
90 :  0.627230703830719
100 :  0.7116076350212097
Test accuracy:  0.9413427114486694

EPOCH 2 of 10

0 :  0.5518532395362854
10 :  0.4142392575740814
20 :  0.5510269999504089
30 :  0.7868683934211731
40 :  0.762485682964325
50 :  0.6681959629058838
60 :  0.7028504014015198
70 :  0.7238516807556152
80 :  0.7524247765541077
90 :  0.34256479144096375
100 :  0.44850486516952515
Test accuracy:  0.9328621625900269

EPOCH 3 of 10

0 :  0.5086467266082764
10 :  0.29810017347335815
20 :  0.5011284351348877
30 :  0.3559761047363281
40 :  0.4948899745941162
50 :  0.28136396408081055
60 :  0.6604571342468262
70 :  0.27999958395957947
80 :  0.5943546891212463
90 :  0.36261841654777527
100 :  0.4927496314048767
Test accuracy:  0.9356890320777893

EPOCH 4 of 10

0 :

In [ ]:
learning_rate = 0.01

model_vgg_77 = vgg16(pretrained=True) 
for param in model_vgg_77.parameters():
    param.requires_grad = False 
    model_vgg_77.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_77.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_77.parameters(), lr=learning_rate)

test_accuracies77 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2, inputs3, labels3) in enumerate(train_loader7):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_77(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_77(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs3.cuda(), labels3.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_77(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_77(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies77.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.4701372385025024
10 :  0.7346718907356262
20 :  0.9538264870643616
30 :  0.6143181920051575
40 :  0.7109382152557373
50 :  0.4383646547794342
60 :  0.6075699329376221
70 :  0.42741477489471436
80 :  0.931001603603363
90 :  0.5762882232666016
100 :  0.3430789113044739
Test accuracy:  0.942756175994873

EPOCH 2 of 10

0 :  0.5362843871116638
10 :  0.6407210230827332
20 :  0.49299848079681396
30 :  0.5057973265647888
40 :  0.2955123484134674
50 :  0.7017301917076111
60 :  0.6431758403778076
70 :  0.5571411848068237
80 :  0.7454109787940979
90 :  0.6711683869361877
100 :  1.0779036283493042
Test accuracy:  0.9187278747558594

EPOCH 3 of 10

0 :  0.8698112964630127
10 :  0.6044859886169434
20 :  0.8259286284446716
30 :  0.5762555599212646
40 :  0.41341763734817505
50 :  0.5748494267463684
60 :  0.6343281865119934
70 :  0.6004152297973633
80 :  0.7571421265602112
90 :  0.8888841867446899
100 :  0.39370882511138916
Test accuracy:  0.9328621625900269

EPOCH 4 of 10

0 : 

In [ ]:
learning_rate = 0.01

model_vgg_88 = vgg16(pretrained=True) 
for param in model_vgg_88.parameters():
    param.requires_grad = False 
    model_vgg_88.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_88.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_88.parameters(), lr=learning_rate)

test_accuracies88 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2, inputs3, labels3) in enumerate(train_loader7):
        inputs_test=torch.cat((inputs1,inputs2,inputs3), 0)
        #print(labels1.size())
        #print(labels2.size())
        labels_test = torch.cat((labels1,labels2,labels3), 0)
        if torch.cuda.is_available():
            inputs, labels = inputs_test.cuda(), labels_test.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_88(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_88(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies88.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.768835425376892
10 :  1.0624781847000122
20 :  0.9842367768287659
30 :  0.8404540419578552
40 :  0.7673830389976501
50 :  0.7417839169502258
60 :  0.7217630743980408
70 :  0.7216701507568359
80 :  0.7568349242210388
90 :  0.8508307337760925
100 :  0.6058545708656311
Test accuracy:  0.9335688948631287

EPOCH 2 of 10

0 :  0.8024097084999084
10 :  0.6675586104393005
20 :  0.6637100577354431
30 :  0.6842494606971741
40 :  0.8953533172607422
50 :  0.7657132744789124
60 :  0.7552620768547058
70 :  0.5606482625007629
80 :  0.6402896046638489
90 :  0.6918928027153015
100 :  0.67555832862854
Test accuracy:  0.9484098553657532

EPOCH 3 of 10

0 :  0.7833402156829834
10 :  0.5096614956855774
20 :  0.6917827129364014
30 :  0.7169898152351379
40 :  0.7097877860069275
50 :  0.6611432433128357
60 :  0.686770498752594
70 :  0.642130434513092
80 :  0.6979994773864746
90 :  0.7877176403999329
100 :  0.617123544216156
Test accuracy:  0.9250882863998413

EPOCH 4 of 10

0 :  0.68875

In [ ]:
learning_rate = 0.01

model_vgg_1 = vgg16(pretrained=True) 
for param in model_vgg_1.parameters():
    param.requires_grad = False 
    model_vgg_1.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_1.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_1.parameters(), lr=learning_rate)

test_accuracies1 = []
epochs = 5
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs, labels) in enumerate(train_loader1):
        # print('test')
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()

        optimizer.zero_grad()
        outputs = model_vgg_1(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):   
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_1(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies1.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 5

0 :  1.6899335384368896
10 :  1.2816120386123657
20 :  0.6955865621566772
30 :  1.0365914106369019
40 :  0.8396865129470825
50 :  0.620980441570282
60 :  0.7674858570098877
70 :  0.7308094501495361
80 :  0.832892656326294
Test accuracy:  0.8197879791259766

EPOCH 2 of 5

0 :  0.7024837732315063
10 :  0.7635772228240967
20 :  0.7106208205223083
30 :  0.6862512826919556
40 :  0.763828694820404
50 :  1.0950392484664917
60 :  0.8795415163040161
70 :  0.8514173626899719
80 :  0.5873005390167236
Test accuracy:  0.8106006979942322

EPOCH 3 of 5

0 :  0.7047269344329834
10 :  0.5216173529624939
20 :  0.7733716368675232
30 :  0.5216346383094788
40 :  0.5584138035774231
50 :  0.7666654586791992
60 :  0.7078908681869507
70 :  0.7033248543739319
80 :  0.7235659956932068
Test accuracy:  0.8459363579750061

EPOCH 4 of 5

0 :  0.7830100655555725
10 :  0.8615654706954956
20 :  0.9173884987831116
30 :  0.8673700094223022
40 :  0.6267516613006592
50 :  0.8925977349281311
60 :  0.703117549

In [ ]:
learning_rate = 0.01

model_vgg_2 = vgg16(pretrained=True) 
for param in model_vgg_2.parameters():
    param.requires_grad = False 
    model_vgg_2.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_2.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_2.parameters(), lr=learning_rate)

test_accuracies2 = []
epochs = 5
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs, labels) in enumerate(train_loader2):
        # print('test')
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()

        optimizer.zero_grad()
        outputs = model_vgg_2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):   
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_2(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies2.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 5

0 :  1.904619812965393
10 :  0.9180461168289185
20 :  0.9642888903617859
30 :  0.41115325689315796
40 :  0.9731889963150024
50 :  0.7681357860565186
60 :  0.7040090560913086
70 :  0.7542816400527954
80 :  0.8236602544784546
90 :  0.5287826657295227
100 :  0.9144055843353271
Test accuracy:  0.7477031350135803

EPOCH 2 of 5

0 :  0.9970888495445251
10 :  0.6063997745513916
20 :  0.7185638546943665
30 :  0.7193825244903564
40 :  0.6711893081665039
50 :  0.3016184866428375
60 :  0.8766108155250549
70 :  0.5857529640197754
80 :  0.44835638999938965
90 :  1.1286317110061646
100 :  0.5637708902359009
Test accuracy:  0.7215547561645508

EPOCH 3 of 5

0 :  0.7419030070304871
10 :  0.6479510068893433
20 :  0.6157661080360413
30 :  0.41064170002937317
40 :  0.46670934557914734
50 :  0.4879299998283386
60 :  0.2991698682308197
70 :  1.0015840530395508
80 :  0.4304351508617401
90 :  0.6042519807815552
100 :  0.5861798524856567
Test accuracy:  0.6706713438034058

EPOCH 4 of 5

0 :  0.

In [ ]:
learning_rate = 0.01

model_vgg_3 = vgg16(pretrained=True) 
for param in model_vgg_3.parameters():
    param.requires_grad = False 
    model_vgg_3.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_3.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_3.parameters(), lr=learning_rate)

test_accuracies3 = []
epochs = 5
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs, labels) in enumerate(train_loader3):
        # print('test')
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()

        optimizer.zero_grad()
        outputs = model_vgg_3(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):   
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_3(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies3.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 5

0 :  1.7876200675964355
10 :  1.2572546005249023
20 :  0.9352366328239441
30 :  0.913779079914093
40 :  0.6740211248397827
50 :  0.5432767868041992
60 :  0.4497413635253906
70 :  0.6562125086784363
80 :  0.46659454703330994
90 :  0.5556900501251221
100 :  0.7677872180938721
Test accuracy:  0.9307420253753662

EPOCH 2 of 5

0 :  0.5606228113174438
10 :  0.4574093818664551
20 :  0.7285712957382202
30 :  0.44280171394348145
40 :  0.6785907745361328
50 :  0.6797785758972168
60 :  0.768382728099823
70 :  0.34215888381004333
80 :  0.5157343149185181
90 :  0.728690505027771
100 :  0.35698556900024414
Test accuracy:  0.9477031826972961

EPOCH 3 of 5

0 :  0.6134851574897766
10 :  0.5967549085617065
20 :  0.3924369513988495
30 :  0.4566679894924164
40 :  0.400419145822525
50 :  0.5227586627006531
60 :  0.40242210030555725
70 :  0.3384259045124054
80 :  0.5167710781097412
90 :  0.31371039152145386
100 :  0.5945956110954285
Test accuracy:  0.9385159015655518

EPOCH 4 of 5

0 :  0.3

In [ ]:
learning_rate = 0.01

model_vgg_4 = vgg16(pretrained=True) 
for param in model_vgg_4.parameters():
    param.requires_grad = False 
    model_vgg_4.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_4.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_4.parameters(), lr=learning_rate)

test_accuracies4 = []
epochs = 5
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2) in enumerate(train_loader4):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_4(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_4(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_4(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies4.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 5

0 :  1.8211697340011597
10 :  0.7903973460197449
20 :  0.7298691868782043
30 :  0.549610435962677
40 :  0.7886589169502258
50 :  0.6889064908027649
60 :  0.9461336731910706
70 :  0.7164089679718018
80 :  0.9891895055770874
90 :  0.7716478705406189
100 :  0.6999116539955139
Test accuracy:  0.8091872334480286

EPOCH 2 of 5

0 :  0.8452547192573547
10 :  0.6170467138290405
20 :  0.8252849578857422
30 :  0.8460256457328796
40 :  0.9119582772254944
50 :  0.5912937521934509
60 :  0.760004997253418
70 :  0.795443058013916
80 :  0.7714139223098755
90 :  1.1058465242385864
100 :  0.7034651041030884
Test accuracy:  0.7406360507011414

EPOCH 3 of 5

0 :  0.8255914449691772
10 :  0.9109963178634644
20 :  0.8518372178077698
30 :  0.5387701392173767
40 :  0.5594547986984253
50 :  0.6250485777854919
60 :  0.6228514313697815
70 :  0.6877089142799377
80 :  0.8132531046867371
90 :  0.5878547430038452
100 :  0.7752541303634644
Test accuracy:  0.7724381685256958

EPOCH 4 of 5

0 :  0.571957

In [ ]:
learning_rate = 0.01

model_vgg_5 = vgg16(pretrained=True) 
for param in model_vgg_5.parameters():
    param.requires_grad = False 
    model_vgg_5.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_5.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_5.parameters(), lr=learning_rate)

test_accuracies5 = []
epochs = 5
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2) in enumerate(train_loader5):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_5(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_5(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_5(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies5.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 5

0 :  1.3589648008346558
10 :  0.965441107749939
20 :  0.7841015458106995
30 :  0.7591086030006409
40 :  0.7396825551986694
50 :  0.954751193523407
60 :  0.5630354881286621
70 :  0.8456906676292419
80 :  0.4787504971027374
90 :  0.8280284404754639
100 :  0.9395500421524048
Test accuracy:  0.9498233199119568

EPOCH 2 of 5

0 :  0.7661314606666565
10 :  0.29633352160453796
20 :  0.47608911991119385
30 :  0.6598268747329712
40 :  0.7964411973953247
50 :  0.7412186861038208
60 :  0.614661693572998
70 :  0.7228685617446899
80 :  0.4964403510093689
90 :  0.589229166507721
100 :  0.7506894469261169
Test accuracy:  0.9335688948631287

EPOCH 3 of 5

0 :  0.5953244566917419
10 :  0.8511752486228943
20 :  0.5021551251411438
30 :  0.6298244595527649
40 :  0.5095137357711792
50 :  0.49160730838775635
60 :  0.5849584937095642
70 :  0.37325429916381836
80 :  0.5707854628562927
90 :  0.5341886878013611
100 :  0.5140775442123413
Test accuracy:  0.9448763132095337

EPOCH 4 of 5

0 :  0.700

In [ ]:
learning_rate = 0.01

model_vgg_6 = vgg16(pretrained=True) 
for param in model_vgg_6.parameters():
    param.requires_grad = False 
model_vgg_6.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_6.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_6.parameters(), lr=learning_rate)

test_accuracies6 = []
epochs = 5
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2) in enumerate(train_loader6):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_6(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_6(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_6(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies6.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 5

0 :  1.4394224882125854
10 :  0.8178913593292236
20 :  0.9281555414199829
30 :  0.7386615872383118
40 :  0.645473837852478
50 :  0.7363678216934204
60 :  0.5464180707931519
70 :  0.5531127452850342
80 :  0.5411671996116638
90 :  0.6601333022117615
100 :  0.5838367342948914
Test accuracy:  0.9399293065071106

EPOCH 2 of 5

0 :  0.3803562819957733
10 :  0.6607180237770081
20 :  0.8909153342247009
30 :  0.6368209719657898
40 :  0.9281451106071472
50 :  0.30110499262809753
60 :  0.6204385161399841
70 :  0.5691832304000854
80 :  0.6077940464019775
90 :  0.7241007685661316
100 :  0.3593682646751404
Test accuracy:  0.9187278747558594

EPOCH 3 of 5

0 :  0.6291812658309937
10 :  0.6884680986404419
20 :  0.4989260137081146
30 :  0.43329960107803345
40 :  0.8784642219543457
50 :  0.630058228969574
60 :  0.7218023538589478
70 :  0.6443032026290894
80 :  0.620275616645813
90 :  0.32631388306617737
100 :  0.674728512763977
Test accuracy:  0.9469964504241943

EPOCH 4 of 5

0 :  0.3367

In [ ]:
learning_rate = 0.01

model_vgg_7 = vgg16(pretrained=True) 
for param in model_vgg_7.parameters():
    param.requires_grad = False 
    model_vgg_7.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_7.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_7.parameters(), lr=learning_rate)

test_accuracies7 = []
epochs = 5
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2, inputs3, labels3) in enumerate(train_loader7):
        if torch.cuda.is_available():
            inputs, labels = inputs1.cuda(), labels1.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_7(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs2.cuda(), labels2.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_7(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if torch.cuda.is_available():
            inputs, labels = inputs3.cuda(), labels3.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_7(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_7(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies7.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 5

0 :  1.4119112491607666
10 :  0.8245828151702881
20 :  0.7070663571357727
30 :  0.5809028148651123
40 :  0.9433038234710693
50 :  0.7987750768661499
60 :  0.5834901928901672
70 :  0.9333563446998596
80 :  0.5168428421020508
90 :  0.4332900047302246
100 :  0.6447421908378601
Test accuracy:  0.9406360387802124

EPOCH 2 of 5

0 :  0.6386478543281555
10 :  0.5661812424659729
20 :  0.5676752924919128
30 :  0.8193607926368713
40 :  0.9627662897109985
50 :  0.497952938079834
60 :  0.5633246898651123
70 :  0.5841193199157715
80 :  0.49541857838630676
90 :  0.6140679121017456
100 :  0.7298353314399719
Test accuracy:  0.9505299925804138

EPOCH 3 of 5

0 :  0.8100401163101196
10 :  0.6920531988143921
20 :  0.5260353088378906
30 :  0.703516960144043
40 :  0.8520663976669312
50 :  0.50505530834198
60 :  0.343456894159317
70 :  0.7062699198722839
80 :  0.32912325859069824
90 :  0.5354326367378235
100 :  0.5623108744621277
Test accuracy:  0.9300352931022644

EPOCH 4 of 5

0 :  0.635554

In [ ]:
learning_rate = 0.01

model_vgg_8 = vgg16(pretrained=True) 
for param in model_vgg_8.parameters():
    param.requires_grad = False 
    model_vgg_8.classifier[6] = nn.Linear(4096, 5)

if torch.cuda.is_available():
    model_vgg_8.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_vgg_8.parameters(), lr=learning_rate)

test_accuracies8 = []
epochs = 10
for epoch in range(epochs):
    print("\nEPOCH " + str(epoch + 1) + " of " + str(epochs) + "\n")

    for i, (inputs1, labels1, inputs2, labels2, inputs3, labels3) in enumerate(train_loader7):
        inputs_test=torch.cat((inputs1,inputs2,inputs3),0)
        #print(labels1.size())
        #print(labels2.size())
        labels_test=torch.cat((labels1,labels2,labels3),0)
        if torch.cuda.is_available():
            inputs, labels = inputs_test.cuda(), labels_test.cuda()
            inputs = inputs.repeat(1, 1, 1, 1).cuda()
        optimizer.zero_grad()
        outputs = model_vgg_8(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 10 == 0:
            print(i, ": ", loss.item())

    test_accuracy = 0 
    for i, (test_inputs, test_labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
            test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
        test_output = model_vgg_8(test_inputs)
        _, test_output = torch.max(test_output, 1)
        test_accuracy += torch.sum(test_output == test_labels)
    test_accuracy = test_accuracy / len(test_loader.dataset)
    test_accuracies8.append(test_accuracy)
    print("Test accuracy: ", test_accuracy.item())


EPOCH 1 of 10

0 :  1.7027446031570435
10 :  1.0368133783340454
20 :  0.9958717823028564
30 :  0.7924929261207581
40 :  0.8287162780761719
50 :  0.7360575199127197
60 :  0.6484932899475098
70 :  0.6689155101776123
80 :  0.7348294854164124
90 :  0.7674890160560608
100 :  0.6353725790977478
Test accuracy:  0.9342755675315857

EPOCH 2 of 10

0 :  0.7456223368644714
10 :  0.8498480916023254
20 :  0.6827771067619324
30 :  0.5190430283546448
40 :  0.847644031047821
50 :  0.717907190322876
60 :  0.6843665242195129
70 :  0.52757328748703
80 :  0.5038631558418274
90 :  0.7110902667045593
100 :  0.6306402087211609
Test accuracy:  0.9286218881607056

EPOCH 3 of 10

0 :  0.7513694763183594
10 :  0.5112013816833496
20 :  0.8319286704063416
30 :  0.6032132506370544
40 :  0.5844742059707642
50 :  0.7539105415344238
60 :  0.6318870186805725
70 :  0.5600712895393372
80 :  0.5759120583534241
90 :  0.7057145237922668
100 :  0.5369864702224731
Test accuracy:  0.9371024370193481

EPOCH 4 of 10

0 :  0.637

BAGGING

In [ ]:
test_accuracy=0
for i, (test_inputs, test_labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
        test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
    test_output = model_vgg_11(test_inputs)
    _, test_output1 = torch.max(test_output, 1)
    # print(test_output1)
    test_output = model_vgg_22(test_inputs)
    _, test_output2 = torch.max(test_output, 1)
    # print(test_output2)
    test_output = model_vgg_33(test_inputs)
    _, test_output3 = torch.max(test_output, 1)
    test_output = model_vgg_44(test_inputs)
    _, test_output4 = torch.max(test_output, 1)
    test_output = model_vgg_55(test_inputs)
    _, test_output5 = torch.max(test_output, 1)
    test_output = model_vgg_66(test_inputs)
    _, test_output6 = torch.max(test_output, 1)
    test_output = model_vgg_77(test_inputs)
    _, test_output7 = torch.max(test_output, 1)
    test_output = model_vgg_88(test_inputs)
    _, test_output8 = torch.max(test_output, 1)
    test_output_all = [test_output1, test_output2, test_output3, test_output4, test_output5, test_output6, test_output7, test_output8]
    # print(test_accuracies1)
    result = torch.tensor([item.cpu().detach().numpy() for item in test_output_all]).cuda()
    result_mode = torch.mode(result, 0).values
    test_accuracy += torch.sum(result_mode == test_labels)
    if i % 10 == 0:
      print(i, test_accuracy)
test_accuracy_bagging = test_accuracy / len(test_loader.dataset)
#test_accuracies7.append(test_accuracy)
print("Test accuracy(Ensemble learning): ", test_accuracy_bagging.item())

0 tensor(31, device='cuda:0')
10 tensor(327, device='cuda:0')
20 tensor(631, device='cuda:0')
30 tensor(930, device='cuda:0')
40 tensor(1231, device='cuda:0')
Test accuracy(Ensemble learning):  0.9363957643508911


Weighted Bagging

In [ ]:
test_accuracy_2 = 0
for i, (test_inputs, test_labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        test_inputs, test_labels = test_inputs.cuda(), test_labels.cuda()
        test_inputs = test_inputs.repeat(1, 1, 1, 1).cuda()
    test_output = model_vgg_1(test_inputs)
    _, test_output1 = torch.max(test_output, 1)
    test_output = model_vgg_2(test_inputs)
    _, test_output2 = torch.max(test_output, 1)
    test_output = model_vgg_3(test_inputs)
    _, test_output3 = torch.max(test_output, 1)
    test_output = model_vgg_4(test_inputs)
    _, test_output4 = torch.max(test_output, 1)
    test_output = model_vgg_5(test_inputs)
    _, test_output5 = torch.max(test_output, 1)
    test_output = model_vgg_6(test_inputs)
    _, test_output6 = torch.max(test_output, 1)
    test_output = model_vgg_7(test_inputs)
    _, test_output7 = torch.max(test_output, 1)
    test_output = model_vgg_8(test_inputs)
    _, test_output8 = torch.max(test_output, 1)
    test_output_all = [test_output1, test_output2, test_output3, test_output4,test_output4,test_output5,test_output5,test_output6,test_output6,test_output7,test_output7,test_output7,test_output8,test_output8,test_output8]
    result = torch.tensor([item.cpu().detach().numpy() for item in test_output_all]).cuda()
    result_mode = torch.mode(result, 0).values
    test_accuracy_2 += torch.sum(result_mode == test_labels)
    if i % 10 == 0:
      print(i, test_accuracy_2)
test_accuracy_bagging_2 = test_accuracy_2 / len(test_loader.dataset)
#test_accuracies7.append(test_accuracy)
print("Test accuracy(Ensemble learning): ", test_accuracy_bagging_2.item())

0 tensor(31, device='cuda:0')
10 tensor(337, device='cuda:0')
20 tensor(642, device='cuda:0')
30 tensor(950, device='cuda:0')
40 tensor(1247, device='cuda:0')
Test accuracy(Ensemble learning):  0.9484098553657532
